In [52]:
import sys
sys.path.append('../src')

from utils import project_root
projroot = project_root()
import warnings
warnings.filterwarnings("ignore")

In [56]:
from models import ToyModel, WhistleNet, CCNN
import yaml
from omegaconf import OmegaConf
from matplotlib import pyplot as plt
from utils import num_params

# Load the configuration
file_path = "ccnn/config.yaml"
with open(file_path, 'r') as file:
    config = OmegaConf.create(yaml.safe_load(file))

name = "whistle/boh"
torch_model = WhistleNet(name)
torch_model.load(name)
torch_input = torch_model.example_input[0]
name = "boh"
torch_out = torch_model(torch_input)
print(torch_out)

MODEL LOADED!
tensor([[3.3871, 0.0000]], grad_fn=<ReluBackward0>)


In [49]:
# from core import ExecutorchModel
from edge import ONNXModel
import onnx
import torch

torch_model.eval()

torch.onnx.export(
    torch_model,
    torch_input,
    f"{name}.onnx",
    verbose=True,
    input_names=["input"],
    output_names=["output"],
)

onnx_model = onnx.load(f"{name}.onnx")
onnx.checker.check_model(onnx_model)

Exported graph: graph(%input : Float(1, 1, 513, strides=[513, 513, 1], requires_grad=0, device=cpu),
      %conv.15.weight : Float(64, 128, 5, strides=[640, 5, 1], requires_grad=1, device=cpu),
      %conv.15.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %fc.0.weight : Float(2, 3520, strides=[3520, 1], requires_grad=1, device=cpu),
      %fc.0.bias : Float(2, strides=[1], requires_grad=1, device=cpu),
      %onnx::Conv_115 : Float(32, 1, 5, strides=[5, 5, 1], requires_grad=0, device=cpu),
      %onnx::Conv_116 : Float(32, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_118 : Float(64, 32, 5, strides=[160, 5, 1], requires_grad=0, device=cpu),
      %onnx::Conv_119 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_121 : Float(128, 64, 5, strides=[320, 5, 1], requires_grad=0, device=cpu),
      %onnx::Conv_122 : Float(128, strides=[1], requires_grad=0, device=cpu)):
  %onnx::Conv_26 : Float(1, 1, 513, strides=[513, 513, 1], requires_

In [50]:
import numpy as np
import onnxruntime

ort_session = onnxruntime.InferenceSession(f"{name}.onnx", providers=["CPUExecutionProvider"])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(torch_input)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!


In [51]:
import time
import numpy as np

onnx_elapsed = 0
iterations = 10000
onnx_elapsed = []

for i in range(iterations):
    torch_input = torch.rand(torch_model.example_input[0].shape)
    ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(torch_input)}
    start = time.time()
    ort_session.run(None, ort_inputs)
    end = time.time()
    onnx_elapsed.append(end - start)

print(np.mean(onnx_elapsed) * 1000)

0.11729550361633301
